In [1]:
import pandas as pd
import numpy as np

In [2]:
res15_valid = pd.read_csv('../probs_subs/check_res15_updated_file.txt')

res26_valid = pd.read_csv('../probs_subs/check_res26_updated_file.txt')

res8_valid = pd.read_csv('../probs_subs/check_res8_updated_file.txt')

In [3]:
res15_valid.sort_values(by='fname', inplace=True)
res26_valid.sort_values(by='fname', inplace=True)
res8_valid.sort_values(by='fname', inplace=True)

In [ ]:
res15_valid.sort_values(by='fname', inplace=True)
res26_valid.sort_values(by='fname', inplace=True)
res8_valid.sort_values(by='fname', inplace=True)

prob_columns = [col for col in res15_valid.columns if 'prob' in col]

res15_valid[prob_columns] = res15_valid[prob_columns].apply(softmax)
res26_valid[prob_columns] = res26_valid[prob_columns].apply(softmax)
res8_valid[prob_columns] = res8_valid[prob_columns].apply(softmax)



In [4]:
def softmax(x):
    scoreMatExp = np.exp(np.asarray(x))
    return scoreMatExp / scoreMatExp.sum(0)

In [5]:
prob_columns = [col for col in res15_valid.columns if 'prob' in col]

res15_valid[prob_columns] = res15_valid[prob_columns].apply(softmax)
res26_valid[prob_columns] = res26_valid[prob_columns].apply(softmax)
res8_valid[prob_columns] = res8_valid[prob_columns].apply(softmax)

In [6]:
res15_valid['cat'] = [s[-2] for s in res15_valid['fname'].str.rsplit('/')]
res15_valid['hash'] = [s[-1] for s in res15_valid['fname'].str.rsplit('/')]

res15_valid['fname'] = res15_valid['cat'] + '/' + res15_valid['hash']

In [7]:
res26_valid['cat'] = [s[-2] for s in res26_valid['fname'].str.rsplit('/')]
res26_valid['hash'] = [s[-1] for s in res26_valid['fname'].str.rsplit('/')]

res26_valid['fname'] = res26_valid['cat'] + '/' + res26_valid['hash']

res8_valid['cat'] = [s[-2] for s in res8_valid['fname'].str.rsplit('/')]
res8_valid['hash'] = [s[-1] for s in res8_valid['fname'].str.rsplit('/')]

res8_valid['fname'] = res8_valid['cat'] + '/' + res8_valid['hash']

In [8]:
res15_valid['cat'].value_counts()

up        272
left      267
off       262
right     259
yes       256
down      253
no        252
go        251
stop      249
on        246
eight      53
five       40
seven      40
one        39
six        38
zero       37
bed        37
three      36
four       34
sheila     33
house      33
nine       32
wow        30
two        30
happy      26
dog        23
cat        22
marvin     21
bird       20
tree       17
Name: cat, dtype: int64

In [9]:
(res26_valid['fname'] != res8_valid['fname'] ).sum()

0

In [10]:
(res15_valid['fname'] != res8_valid['fname'] ).sum()

0

In [11]:
res26_valid.drop(['fname', 'label'], axis=1, inplace=True)
res8_valid.drop(['fname', 'label'], axis=1, inplace=True)

In [12]:
res26_valid['max_prob'] = res26_valid[prob_columns].idxmax(axis=1).str.extract('(\d+)').astype(int)
res8_valid['max_prob'] = res8_valid[prob_columns].idxmax(axis=1).str.extract('(\d+)').astype(int)
res15_valid['max_prob'] = res15_valid[prob_columns].idxmax(axis=1).str.extract('(\d+)').astype(int)

/home/egor/tensorflow/lib/python3.5/site-packages/ipykernel_launcher.py:1: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
  """Entry point for launching an IPython kernel.
/home/egor/tensorflow/lib/python3.5/site-packages/ipykernel_launcher.py:2: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
  
/home/egor/tensorflow/lib/python3.5/site-packages/ipykernel_launcher.py:3: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
  This is separate from the ipykernel package so we can avoid doing imports until


In [13]:
label2word = {0: 'silence',
 1: 'unknown',
 2: 'yes',
 3: 'no',
 4: 'up',
 5: 'down',
 6: 'left',
 7: 'right',
 8: 'on',
 9: 'off',
 10: 'stop',
 11: 'go'}

In [14]:
res26_valid['max_prob_label'] = res26_valid['max_prob'].map(label2word)
res8_valid['max_prob_label'] = res8_valid['max_prob'].map(label2word)
res15_valid['max_prob_label'] = res15_valid['max_prob'].map(label2word)

In [15]:
res15_valid.columns = ['res15_' + col for col in res15_valid.columns]
res8_valid.columns = ['res8_' + col for col in res8_valid.columns]
res26_valid.columns = ['res26_' + col for col in res26_valid.columns]

In [16]:
common_valid = pd.concat([res8_valid, res26_valid, res15_valid], axis=1)

In [17]:
common_valid['true_label'] = common_valid['res15_label'].map(label2word)

In [18]:
voting = common_valid[[col for col in common_valid.columns if ('label' in col) and ('res15_label' not in col)]]

In [19]:
voting[(voting.res15_max_prob_label != voting.true_label) &
       ((voting.res15_max_prob_label == voting.true_label) |
       (voting.res15_max_prob_label == voting.true_label))]

,res8_max_prob_label,res26_max_prob_label,res15_max_prob_label,true_label


In [20]:
common_valid.columns

Index(['res8_prob_0', 'res8_prob_1', 'res8_prob_2', 'res8_prob_3',
       'res8_prob_4', 'res8_prob_5', 'res8_prob_6', 'res8_prob_7',
       'res8_prob_8', 'res8_prob_9', 'res8_prob_10', 'res8_prob_11',
       'res8_cat', 'res8_hash', 'res8_max_prob', 'res8_max_prob_label',
       'res26_prob_0', 'res26_prob_1', 'res26_prob_2', 'res26_prob_3',
       'res26_prob_4', 'res26_prob_5', 'res26_prob_6', 'res26_prob_7',
       'res26_prob_8', 'res26_prob_9', 'res26_prob_10', 'res26_prob_11',
       'res26_cat', 'res26_hash', 'res26_max_prob', 'res26_max_prob_label',
       'res15_fname', 'res15_prob_0', 'res15_prob_1', 'res15_prob_2',
       'res15_prob_3', 'res15_prob_4', 'res15_prob_5', 'res15_prob_6',
       'res15_prob_7', 'res15_prob_8', 'res15_prob_9', 'res15_prob_10',
       'res15_prob_11', 'res15_label', 'res15_cat', 'res15_hash',
       'res15_max_prob', 'res15_max_prob_label', 'true_label'],
      dtype='object')

In [21]:
for prob in range(12):
    gmean = (common_valid['res8_prob_' + str(prob)] * \
                                         common_valid['res26_prob_' + str(prob)] * \
                                         common_valid['res15_prob_' + str(prob)])**(1/3)
    common_valid['gmean_' + str(prob)] = gmean

common_valid['gmean_max_prob_label'] = common_valid[[col for col in common_valid.columns if 'gmean' in col]] \
                                                                                .idxmax(axis=1) \
                                                                                .str.extract('(\d+)') \
                                                                                .astype(int) \
                                                                                .map(label2word)

/home/egor/tensorflow/lib/python3.5/site-packages/ipykernel_launcher.py:5: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
  """


In [22]:
for prob in range(12):
    gmean = (common_valid['res8_prob_' + str(prob)]**2 * \
                                         common_valid['res26_prob_' + str(prob)]**2 * \
                                         common_valid['res15_prob_' + str(prob)]**2)**(1/3)
    common_valid['gmean_squared' + str(prob)] = gmean

common_valid['gmean_squared_max_prob_label'] = common_valid[[col for col in common_valid.columns if 'gmean_squared' in col]] \
                                                                                .idxmax(axis=1) \
                                                                                .str.extract('(\d+)') \
                                                                                .astype(int) \
                                                                                .map(label2word)

/home/egor/tensorflow/lib/python3.5/site-packages/ipykernel_launcher.py:5: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
  """


In [23]:
for prob in range(12):
    gmean = (common_valid['res8_prob_' + str(prob)]**2 * \
                                         common_valid['res26_prob_' + str(prob)]**2 * \
                                         common_valid['res15_prob_' + str(prob)])**(1/3)
    common_valid['gmean_squared_ex_res_fifthteen' + str(prob)] = gmean

common_valid['gmean_squared_ex_res_fifthteen_max_prob_label'] = common_valid[[col for col in common_valid.columns if 'gmean_squared_ex_res_fifthteen' in col]] \
                                                                                .idxmax(axis=1) \
                                                                                .str.extract('(\d+)') \
                                                                                .astype(int) \
                                                                                .map(label2word)

/home/egor/tensorflow/lib/python3.5/site-packages/ipykernel_launcher.py:5: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
  """


In [24]:
for prob in range(12):
    mean = (common_valid['res8_prob_' + str(prob)] + \
                                         common_valid['res26_prob_' + str(prob)] + \
                                         common_valid['res15_prob_' + str(prob)])/3
    common_valid['usual_mean' + str(prob)] = mean

common_valid['mean_max_prob_label'] = common_valid[[col for col in common_valid.columns if 'usual_mean' in col]] \
                                                                                .idxmax(axis=1) \
                                                                                .str.extract('(\d+)') \
                                                                                .astype(int) \
                                                                                .map(label2word)

/home/egor/tensorflow/lib/python3.5/site-packages/ipykernel_launcher.py:5: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
  """


In [25]:
np.arange(0, 0.5, 0.05)

array([ 0.  ,  0.05,  0.1 ,  0.15,  0.2 ,  0.25,  0.3 ,  0.35,  0.4 ,  0.45])

In [26]:
find_coef = common_valid[[col for col in common_valid.columns if ('res8_prob_' in col) |
                                                                 ('res26_prob_' in col) |
                                                                 ('res15_prob_' in col) |
                                                                 ('true_label' in col)]]

In [27]:
res15_coef = 0.99
others_coef = 1 - res15_coef

for i, coef in enumerate(np.arange(0, others_coef, 0.0005)):
    for prob in range(12):
        other_coef = others_coef - coef
        
        mean = find_coef['res8_prob_' + str(prob)] * coef + \
                find_coef['res26_prob_' + str(prob)] * other_coef + \
                find_coef['res15_prob_' + str(prob)] * res15_coef
        find_coef['coefs_mean' + str(prob)] = mean
        
    find_coef['try_N' + str(i)] = find_coef[[col for col in find_coef.columns if 'coefs_mean' in col]] \
                                                                                    .idxmax(axis=1) \
                                                                                    .str.extract('(\d+)') \
                                                                                    .astype(int) \
                                                                                    .map(label2word)

/home/egor/tensorflow/lib/python3.5/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/home/egor/tensorflow/lib/python3.5/site-packages/ipykernel_launcher.py:11: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
  # This is added back by InteractiveShellApp.init_path()
/home/egor/tensorflow/lib/python3.5/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stabl

In [ ]:
# 0.4 res15
for guess in [col for col in find_coef.columns if 'try_N' in col]:   
    print(find_coef[find_coef[guess] == find_coef['true_label']].shape)

In [ ]:
# 0.5 res15
for guess in [col for col in find_coef.columns if 'try_N' in col]:   
    print(find_coef[find_coef[guess] == find_coef['true_label']].shape)

In [ ]:
# 0.6 res15
for guess in [col for col in find_coef.columns if 'try_N' in col]:   
    print(find_coef[find_coef[guess] == find_coef['true_label']].shape)

In [ ]:
# 0.9 res15
for guess in [col for col in find_coef.columns if 'try_N' in col]:   
    print(find_coef[find_coef[guess] == find_coef['true_label']].shape)

In [27]:
# 0.95 res15
for guess in [col for col in find_coef.columns if 'try_N' in col]:   
    print(find_coef[find_coef[guess] == find_coef['true_label']].shape)

(2228, 55)
(2108, 55)
(2073, 55)
(2088, 55)
(2121, 55)
(2251, 55)


In [27]:
# 0.97 res15
for guess in [col for col in find_coef.columns if 'try_N' in col]:   
    print(find_coef[find_coef[guess] == find_coef['true_label']].shape)

(2322, 56)
(2222, 56)
(2201, 56)
(2195, 56)
(2200, 56)
(2238, 56)
(2347, 56)


In [27]:
# 0.99 res15
for guess in [col for col in find_coef.columns if 'try_N' in col]:   
    print(find_coef[find_coef[guess] == find_coef['true_label']].shape)

(2507, 70)
(2462, 70)
(2448, 70)
(2435, 70)
(2424, 70)
(2418, 70)
(2412, 70)
(2414, 70)
(2410, 70)
(2409, 70)
(2408, 70)
(2417, 70)
(2419, 70)
(2422, 70)
(2424, 70)
(2430, 70)
(2432, 70)
(2445, 70)
(2454, 70)
(2464, 70)
(2507, 70)


In [29]:
common_valid.columns

Index(['res8_prob_0', 'res8_prob_1', 'res8_prob_2', 'res8_prob_3',
       'res8_prob_4', 'res8_prob_5', 'res8_prob_6', 'res8_prob_7',
       'res8_prob_8', 'res8_prob_9',
       ...
       'usual_mean3', 'usual_mean4', 'usual_mean5', 'usual_mean6',
       'usual_mean7', 'usual_mean8', 'usual_mean9', 'usual_mean10',
       'usual_mean11', 'mean_max_prob_label'],
      dtype='object', length=103)

In [23]:
common_valid.columns

Index(['res8_prob_0', 'res8_prob_1', 'res8_prob_2', 'res8_prob_3',
       'res8_prob_4', 'res8_prob_5', 'res8_prob_6', 'res8_prob_7',
       'res8_prob_8', 'res8_prob_9',
       ...
       'usual_mean3', 'usual_mean4', 'usual_mean5', 'usual_mean6',
       'usual_mean7', 'usual_mean8', 'usual_mean9', 'usual_mean10',
       'usual_mean11', 'mean_max_prob_label'],
      dtype='object', length=103)

In [28]:
[col for col in common_valid.columns if 'label' in col]

['res8_max_prob_label',
 'res26_max_prob_label',
 'res15_label',
 'res15_max_prob_label',
 'true_label',
 'gmean_max_prob_label',
 'gmean_squared_max_prob_label',
 'gmean_squared_ex_res_fifthteen_max_prob_label',
 'mean_max_prob_label']

In [28]:
common_valid.drop(['res8_max_prob_label',
 'res26_max_prob_label',
 'res15_label',
 'res15_max_prob_label',
 'gmean_max_prob_label',
 'gmean_squared_max_prob_label',
 'gmean_squared_ex_res_fifthteen_max_prob_label',
 'mean_max_prob_label'], axis=1, inplace=True)

In [32]:
common_valid.drop('res15_fname', axis=1, inplace=True)

In [35]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()


In [36]:
[col for col in common_valid.columns if 'hash' in col]

['res8_hash', 'res26_hash', 'res15_hash']

In [40]:
common_valid['res8_hash'] = le.fit_transform(common_valid['res8_hash'])

In [41]:
common_valid['res26_hash'] = le.fit_transform(common_valid['res26_hash'])

In [42]:
common_valid['res15_hash'] = le.fit_transform(common_valid['res15_hash'])

In [47]:
common_valid.drop([col for col in common_valid.columns if 'cat' in col], axis=1, inplace=True)

In [50]:
common_valid.drop([col for col in common_valid.columns if 'hash' in col], axis=1, inplace=True)

In [102]:
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier

from sklearn.model_selection import cross_val_score
clf = RandomForestClassifier(n_estimators=1000)
scores = cross_val_score(clf, common_valid[[col for col in common_valid.columns if 'true_label' not in col]], 
                         common_valid.true_label, cv=3)

In [55]:
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, GradientBoostingClassifier

from sklearn.model_selection import cross_val_score
clf = ExtraTreesClassifier(n_estimators=1000)
scores = cross_val_score(clf, common_valid[[col for col in common_valid.columns if 'true_label' not in col]], 
                         common_valid.true_label, cv=3)

In [30]:
# logistic
scores

array([ 0.89013035,  0.89148737,  0.89389671])

In [54]:
# rf with n_estimators=1000
scores

array([ 0.90595903,  0.91393826,  0.92018779])

In [103]:
# rf with n_estimators=1000
scores

array([ 0.90502793,  0.91206735,  0.92018779])

In [104]:
clf.fit(common_valid[[col for col in common_valid.columns if 'true_label' not in col]], 
                         common_valid.true_label)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=1000, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [56]:
# et with n_estimators=1000
scores

array([ 0.90316574,  0.90739008,  0.91643192])

In [28]:
find_coef[[col for col in find_coef.columns if ('try_N' in col) | ('true_label' in col)]]

,true_label,try_N0,try_N1,try_N2,try_N3,try_N4,try_N5
291,down,no,no,go,go,go,go
1426,no,silence,silence,silence,silence,no,no
2644,unknown,unknown,unknown,yes,yes,yes,yes
576,silence,silence,silence,silence,silence,silence,silence
1144,up,up,up,up,up,up,up
753,no,go,right,right,right,right,right
2150,on,on,on,on,on,on,on
1564,right,right,unknown,unknown,unknown,unknown,unknown
3174,down,down,down,down,down,down,down
2287,unknown,unknown,silence,silence,silence,silence,silence


In [29]:
ensemble = common_valid[['res15_max_prob_label', 'gmean_max_prob_label', 
                         "gmean_squared_ex_res_fifthteen_max_prob_label",
                         'mean_max_prob_label', 'gmean_squared_max_prob_label', 
                         'true_label']]

In [30]:
ensemble[ensemble['res15_max_prob_label'] == ensemble['true_label']].shape

(2894, 6)

In [31]:
ensemble[ensemble['gmean_max_prob_label'] == ensemble['true_label']].shape

(1348, 6)

In [32]:
ensemble[ensemble['mean_max_prob_label'] == ensemble['true_label']].shape

(1213, 6)

In [33]:
ensemble[ensemble['gmean_squared_max_prob_label'] == ensemble['true_label']].shape

(1348, 6)

In [34]:
ensemble[ensemble['gmean_squared_ex_res_fifthteen_max_prob_label'] == ensemble['true_label']].shape

(769, 6)

# Вот тут ЖЁПА

In [35]:
common_valid[(common_valid.res26_max_prob_label == common_valid.true_label)].shape

(313, 103)

In [36]:
common_valid[(common_valid.res8_max_prob_label == common_valid.true_label)].shape

(339, 103)

In [37]:
common_valid[(common_valid.res26_max_prob_label == common_valid.true_label) &
             (common_valid.res15_max_prob_label != common_valid.true_label)].shape

(24, 103)

In [38]:
common_valid[(common_valid.res26_max_prob_label == common_valid.true_label) &
             (common_valid.res15_max_prob_label != common_valid.true_label)][ensemble.columns]

,res15_max_prob_label,gmean_max_prob_label,gmean_squared_ex_res_fifthteen_max_prob_label,mean_max_prob_label,gmean_squared_max_prob_label,true_label
1888,go,unknown,unknown,unknown,unknown,unknown
1009,up,off,off,yes,off,yes
721,no,silence,silence,silence,silence,unknown
372,down,no,no,no,no,no
1974,silence,silence,silence,silence,silence,on
1497,yes,unknown,unknown,unknown,unknown,unknown
2232,stop,unknown,unknown,silence,unknown,unknown
3175,unknown,yes,yes,yes,yes,yes
2486,stop,no,no,no,no,up
2050,up,unknown,unknown,unknown,unknown,unknown


In [39]:
common_valid[(common_valid.res8_max_prob_label == common_valid.true_label) &
             (common_valid.res15_max_prob_label != common_valid.true_label)].shape

(26, 103)

In [40]:
common_valid[((common_valid.res8_max_prob_label == common_valid.true_label) |
             (common_valid.res15_max_prob_label == common_valid.true_label)) &
             (common_valid.res15_max_prob_label != common_valid.true_label)][ensemble.columns]

,res15_max_prob_label,gmean_max_prob_label,gmean_squared_ex_res_fifthteen_max_prob_label,mean_max_prob_label,gmean_squared_max_prob_label,true_label
782,stop,unknown,unknown,right,unknown,unknown
2673,unknown,left,left,right,left,right
2835,yes,no,no,no,no,unknown
1128,silence,left,left,unknown,left,unknown
1888,go,unknown,unknown,unknown,unknown,unknown
1442,off,up,up,up,up,up
3189,unknown,go,go,yes,go,go
2526,silence,off,off,off,off,unknown
3095,stop,unknown,unknown,unknown,unknown,unknown
1385,silence,unknown,unknown,down,unknown,down


In [41]:
common_valid.shape

(3208, 103)

In [42]:
# res15 дает 2894 из 3208
print(2894 / 3208)
print((2894 + 26) / 3208)

0.9021197007481296
0.9102244389027432


негусто, но что-то

In [91]:
res15 = pd.read_csv('../probs_subs/mix_model_res15.txt')

res26 = pd.read_csv('../probs_subs/mix_model_res26.txt')

res8 = pd.read_csv('../probs_subs/mix_model_res8.txt')

In [92]:
res15.sort_values(by='fname', inplace=True)
res26.sort_values(by='fname', inplace=True)
res8.sort_values(by='fname', inplace=True)

prob_columns = [col for col in res15.columns if 'prob' in col]

res15[prob_columns] = res15[prob_columns].apply(softmax)
res26[prob_columns] = res26[prob_columns].apply(softmax)
res8[prob_columns] = res8[prob_columns].apply(softmax)

In [93]:
res26['max_prob'] = res26[prob_columns].idxmax(axis=1).str.extract('(\d+)').astype(int)
res8['max_prob'] = res8[prob_columns].idxmax(axis=1).str.extract('(\d+)').astype(int)
res15['max_prob'] = res15[prob_columns].idxmax(axis=1).str.extract('(\d+)').astype(int)

/home/egor/tensorflow/lib/python3.5/site-packages/ipykernel_launcher.py:1: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
  """Entry point for launching an IPython kernel.
/home/egor/tensorflow/lib/python3.5/site-packages/ipykernel_launcher.py:2: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
  
/home/egor/tensorflow/lib/python3.5/site-packages/ipykernel_launcher.py:3: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
  This is separate from the ipykernel package so we can avoid doing imports until


In [94]:
res15.columns = ['res15_' + col for col in res15.columns]
res8.columns = ['res8_' + col for col in res8.columns]
res26.columns = ['res26_' + col for col in res26.columns]


common_test = pd.concat([res8, res26, res15], axis=1)

In [95]:
for prob in range(12):
    gmean = (common_test['res8_prob_' + str(prob)] * \
                                         common_test['res26_prob_' + str(prob)] * \
                                         common_test['res15_prob_' + str(prob)])**(1/3)
    common_test['gmean_' + str(prob)] = gmean

# common_test['gmean_max_prob_label'] = common_test[[col for col in common_test.columns if 'gmean' in col]] \
#                                                                                 .idxmax(axis=1) \
#                                                                                 .str.extract('(\d+)') \
#                                                                                 .astype(int) \
#                                                                                 .map(label2word)

In [96]:
for prob in range(12):
    gmean = (common_test['res8_prob_' + str(prob)]**2 * \
                                         common_test['res26_prob_' + str(prob)]**2 * \
                                         common_test['res15_prob_' + str(prob)]**2)**(1/3)
    common_test['gmean_squared' + str(prob)] = gmean

# common_test['gmean_squared_max_prob_label'] = common_test[[col for col in common_test.columns if 'gmean_squared' in col]] \
#                                                                                 .idxmax(axis=1) \
#                                                                                 .str.extract('(\d+)') \
#                                                                                 .astype(int) \
#                                                                                 .map(label2word)

In [97]:
for prob in range(12):
    gmean = (common_test['res8_prob_' + str(prob)]**2 * \
                                         common_test['res26_prob_' + str(prob)]**2 * \
                                         common_test['res15_prob_' + str(prob)])**(1/3)
    common_test['gmean_squared_ex_res_fifthteen' + str(prob)] = gmean

# common_test['gmean_squared_ex_res_fifthteen_max_prob_label'] = common_test[[col for col in common_test.columns if 'gmean_squared_ex_res_fifthteen' in col]] \
#                                                                                 .idxmax(axis=1) \
#                                                                                 .str.extract('(\d+)') \
#                                                                                 .astype(int) \
#                                                                                 .map(label2word)

In [98]:
for prob in range(12):
    mean = (common_test['res8_prob_' + str(prob)] + \
                                         common_test['res26_prob_' + str(prob)] + \
                                         common_test['res15_prob_' + str(prob)])/3
    common_test['usual_mean' + str(prob)] = mean

# common_test['mean_max_prob_label'] = common_test[[col for col in common_test.columns if 'usual_mean' in col]] \
#                                                                                 .idxmax(axis=1) \
#                                                                                 .str.extract('(\d+)') \
#                                                                                 .astype(int) \
#                                                                                 .map(label2word)

In [99]:
common_test.shape

(158538, 90)

In [101]:
set(common_valid.columns) - set(common_test.columns) 

{'true_label'}

In [ ]:
clf.fit(common_valid[[col for col in common_valid.columns if 'true_label' not in col]], 
                         common_valid.true_label)

In [108]:
common_test['predictions'] = clf.predict(common_test[[col for col in common_test.columns if \
                                                      ('true_label' not in col) & ('fname' not in col)]])

In [109]:
common_test['predictions'] = 

0         unknown
1              no
2         unknown
3         unknown
4             off
5         unknown
6         unknown
7         unknown
8         unknown
9         silence
10        unknown
11        unknown
12        unknown
13        unknown
14        unknown
15            off
16            off
17            off
18        unknown
19           stop
20        unknown
21        unknown
22           down
23        unknown
24        unknown
25        unknown
26        unknown
27        unknown
28        unknown
29        silence
           ...   
158508    unknown
158509    unknown
158510       down
158511    unknown
158512    unknown
158513       left
158514       stop
158515    unknown
158516        yes
158517         go
158518    unknown
158519       left
158520        off
158521    unknown
158522    unknown
158523    unknown
158524    unknown
158525    unknown
158526         up
158527    unknown
158528    unknown
158529         go
158530    unknown
158531        yes
158532    

In [106]:
common_test.columns

Index(['res8_fname', 'res8_prob_0', 'res8_prob_1', 'res8_prob_2',
       'res8_prob_3', 'res8_prob_4', 'res8_prob_5', 'res8_prob_6',
       'res8_prob_7', 'res8_prob_8', 'res8_prob_9', 'res8_prob_10',
       'res8_prob_11', 'res8_max_prob', 'res26_fname', 'res26_prob_0',
       'res26_prob_1', 'res26_prob_2', 'res26_prob_3', 'res26_prob_4',
       'res26_prob_5', 'res26_prob_6', 'res26_prob_7', 'res26_prob_8',
       'res26_prob_9', 'res26_prob_10', 'res26_prob_11', 'res26_max_prob',
       'res15_fname', 'res15_prob_0', 'res15_prob_1', 'res15_prob_2',
       'res15_prob_3', 'res15_prob_4', 'res15_prob_5', 'res15_prob_6',
       'res15_prob_7', 'res15_prob_8', 'res15_prob_9', 'res15_prob_10',
       'res15_prob_11', 'res15_max_prob', 'gmean_0', 'gmean_1', 'gmean_2',
       'gmean_3', 'gmean_4', 'gmean_5', 'gmean_6', 'gmean_7', 'gmean_8',
       'gmean_9', 'gmean_10', 'gmean_11', 'gmean_squared0', 'gmean_squared1',
       'gmean_squared2', 'gmean_squared3', 'gmean_squared4', 'gmean_squa

In [111]:
common_test[['res8_fname', 'predictions']].to_csv('rf.csv', index=False)

In [ ]:
res15['label'] = res15[[col for col in res15.columns if 'prob' in col]].idxmax(axis=1).str.extract('(\d+)').astype(int)

In [ ]:
res15[['fname', 'label']].to_csv('res15.csv', index=False)

In [ ]:
res15['label'] = res15['label'].map(label2word)

In [ ]:
res15['label'] = res15['label'].map(label2word)

In [ ]:
sol085 = pd.read_csv('../submissions/submission_mix_noises_updated.txt')

In [ ]:
sol085.sort_values(by='fname', inplace=True)
sol085.head()

In [ ]:
res15.sort_values(by='fname', inplace=True)
res15.head()

In [ ]:
res15['085_sol'] = sol085['label']

In [ ]:
res15['sum'] = res15['prob_0'] + res15['prob_1'] + res15['prob_2'] + res15['prob_3'] + res15['prob_4'] + \
               res15['prob_5'] + res15['prob_6'] + res15['prob_7'] + res15['prob_8'] + res15['prob_9'] + \
               res15['prob_10'] + res15['prob_11']

In [ ]:
res15['sum'].describe()

In [ ]:
res26['sum'] = res26['prob_0'] + res26['prob_1'] + res26['prob_2'] + res26['prob_3'] + res26['prob_4'] + \
               res26['prob_5'] + res26['prob_6'] + res26['prob_7'] + res26['prob_8'] + res26['prob_9'] + \
               res26['prob_10'] + res26['prob_11']

In [ ]:
res26['sum'].describe()

In [ ]:
res8['sum'] = res8['prob_0'] + res8['prob_1'] + res8['prob_2'] + res8['prob_3'] + res8['prob_4'] + \
               res8['prob_5'] + res8['prob_6'] + res8['prob_7'] + res8['prob_8'] + res8['prob_9'] + \
               res8['prob_10'] + res8['prob_11']

In [ ]:
res8['sum'].describe()